In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[70]:


import pymysql

conn = pymysql.connect(
    host='  ',
    port=3306,
    user='  ',
    password='  ',
    db='  ',
    charset='utf8'
)
cursor = conn.cursor(pymysql.cursors.DictCursor)


def query(sql):
    global cursor
    cursor.execute(sql)
    result = cursor.fetchall()
    conn.commit()
    return result

#from database import query


# In[71]:



#from database import query
import time
import pandas as pd

# 최신성 스코어 구하기
def getRecencyScore(reviewId):
    global firstReviewTime, lastReviewTime
    maxScore = lastReviewTime - firstReviewTime
    sql = 'SELECT createdAt FROM Reviews WHERE id = {0}'.format(reviewId)
    review = query(sql)

    if review is None or len(review) == 0:
        return 0

    reviewTime = review[0]['createdAt']
    score = reviewTime - firstReviewTime
    return score / maxScore * 50


# 최대 리뷰 좋아요 수 구하기.
def getMaxReviewLike():
    sql = "SELECT COUNT(ReviewLikes.reviewId) FROM Reviews LEFT JOIN ReviewLikes ON ReviewLikes.reviewId = Reviews.id GROUP BY Reviews.id ORDER BY COUNT(ReviewLikes.reviewId) DESC"
    maxReviewLike = query(sql)
    return maxReviewLike[0]['COUNT(ReviewLikes.reviewId)']


# 최대 리뷰 댓글 수 구하기.
def getMaxReviewComment():
    sql = "SELECT COUNT(ReviewComments.reviewId) FROM Reviews LEFT JOIN ReviewComments ON ReviewComments.reviewId = Reviews.id GROUP BY Reviews.id ORDER BY COUNT(ReviewComments.reviewId) DESC"
    maxReviewComment = query(sql)
    return maxReviewComment[0]['COUNT(ReviewComments.reviewId)']


# 최대 유저 팔로잉 수 구하기.
def getMaxFollow():
    sql = "SELECT COUNT(UserFollows.followingId) FROM Users LEFT JOIN UserFollows ON UserFollows.followingId = Users.id GROUP BY Users.id ORDER BY COUNT(UserFollows.followingId) DESC"
    getMaxFollow = query(sql)
    return getMaxFollow[0]['COUNT(UserFollows.followingId)']


# 리뷰의 좋아요 점수 구하기.
def getScoreOfReviewLike(reviewId):
    global maxReviewScore
    sql = "SELECT COUNT(reviewId) FROM ReviewLikes WHERE reviewId = {0}".format(reviewId)
    score = query(sql)
    if score is None or len(score) == 0:
        return 0
    return score[0]['COUNT(reviewId)'] / maxReviewScore * 10


# 리뷰의 댓글 점수 구하기.
def getScoreOfReviewComment(reviewId):
    global maxCommentScore
    sql = "SELECT COUNT(id) FROM ReviewComments WHERE reviewId = {0}".format(reviewId)
    score = query(sql)
    if score is None or len(score) == 0:
        return 0
    return score[0]['COUNT(id)'] / maxCommentScore * 10

# 리뷰의 팔로우 점수 구하기.
def getScoreOfFollow(reviewId):
    global maxFollowScore
    sql = "SELECT COUNT(UserFollows.followerId) FROM UserFollows LEFT JOIN Reviews ON Reviews.authorId = UserFollows.followingId WHERE Reviews.id = {0}".format(reviewId)
    score = query(sql)
    if score is None or len(score) == 0:
        return 0
    return score[0]['COUNT(UserFollows.followerId)'] / maxFollowScore * 10

# 대중성 스코어 구하기
def getPopularityScore(reviewId):
    return getScoreOfReviewLike(reviewId) + getScoreOfReviewComment(reviewId) + getScoreOfFollow(reviewId)

# 최신성 + 대중성 점수
def getTotalScore(reviewId):
    return getRecencyScore(reviewId) + getPopularityScore(reviewId)

# 친밀성 점수 구하기
def getTasteScore(userId, reviewId):
    score = 0
    sql = 'SELECT id FROM Reviews LEFT JOIN UserFollows ON UserFollows.followingId = Reviews.authorId WHERE Reviews.id = {0} and UserFollows.followerId = {1}'.format(reviewId, userId)
    checkFollows = query(sql)
    if len(checkFollows) > 0:
        score += 5

    sql = 'SELECT * FROM BookmarkReviews ' \
          'LEFT JOIN Reviews as authorReview ON BookmarkReviews.reviewId = authorReview.id ' \
          'LEFT JOIN Reviews as seeingReview ON seeingReview.authorId = authorReview.authorId ' \
          'LEFT JOIN Bookmarks ON BookmarkReviews.bookmarkId = Bookmarks.id ' \
          'WHERE Bookmarks.userId = {0} and seeingReview.id = {1}'.format(userId, reviewId)
    checkBookmarks = query(sql)
    if len(checkBookmarks) > 0:
        score += 5

    sql = 'SELECT * FROM ReviewComments ' \
          'LEFT JOIN Reviews as authorReview ON ReviewComments.reviewId = authorReview.id ' \
          'LEFT JOIN Reviews as seeingReview ON seeingReview.authorId = authorReview.authorId ' \
          'WHERE ReviewComments.authorId = {0} and seeingReview.id = {1}'.format(userId, reviewId)
    checkComments = query(sql)
    if len(checkComments) > 0:
        score += 5

    sql = 'SELECT * FROM ReviewLikes ' \
          'LEFT JOIN Reviews as authorReview ON ReviewLikes.reviewId = authorReview.id ' \
          'LEFT JOIN Reviews as seeingReview ON seeingReview.authorId = authorReview.authorId ' \
          'WHERE ReviewLikes.senderId = {0} and seeingReview.id = {1}'.format(userId, reviewId)
    checkLikes = query(sql)
    if len(checkLikes) > 0:
        score += 5

    return score

# 메인 함수.
if __name__ == '__main__':

    maxReviewScore = getMaxReviewLike()
    maxCommentScore = getMaxReviewComment()
    maxFollowScore = getMaxFollow()

    firstReviewTime = query('SELECT createdAt FROM Reviews ORDER BY createdAt ASC LIMIT 1')[0]['createdAt']
    lastReviewTime = query('SELECT createdAt FROM Reviews ORDER BY createdAt DESC LIMIT 1')[0]['createdAt']
    
    
    
def Dataframe_PopularityRecencyScore(): 
    Reviews = pd.read_sql_query('SELECT * FROM Reviews WHERE NOT authorId = 928', conn)
    global Reduced_Reviews
    Reduced_Reviews = Reviews.sample(n=300)
    Reduced_Reviews['Score'] = 0
    for i in Reduced_Reviews['id']:
        PopularityRecencyScore = getTotalScore(i)
        Reduced_Reviews['Score'][Reduced_Reviews['id'] == i] = PopularityRecencyScore
    Reduced_Reviews = Reduced_Reviews.sort_values(by=['Score'], axis=0, ascending=False)
    Reduced_Reviews.reset_index(drop=True, inplace = True)
    Reduced_Reviews = Reduced_Reviews.iloc[0:100]
    Reduced_Reviews['Affinity'] = 0
    return Reduced_Reviews


In [ ]:

Dataframe_PopularityRecencyScore()

In [ ]:
import time
start = time.time() 
def GetTastyScore_EachUsers(userId):
    global Reduced_Reviews20
    Reduced_Reviews20 =  Reduced_Reviews.iloc[0:20]
    Reduced_Reviews20.reset_index(drop=True, inplace = True)
    Reduced_Reviews_Id_List = Reduced_Reviews20['id'].tolist()
    for i in range(len(Reduced_Reviews_Id_List)):
        Reduced_Reviews20['Affinity'][i]=getTasteScore(userId, Reduced_Reviews_Id_List[i])
    Reduced_Reviews20 = Reduced_Reviews20.sort_values(by=['Affinity'], axis=0, ascending=False)
    return Reduced_Reviews20
GetTastyScore_EachUsers(9)
print("time :", time.time() - start) 